# 삼성전자 첨기연 시각 심화

- **Instructor**: Jongwoo Lim / Jiun Bae
- **Email**: [jlim@hanyang.ac.kr](mailto:jlim@hanyang.ac.kr) / [jiun.maydev@gmail.com](mailto:jiun.maydev@gmail.com)

### Import packages

First of all, Import some packages for using PyTorch.

- torch.nn: The **Network** of PyTorch basically starts with nn.Module.
- torch.nn.functional: for **Functions** such as *ReLU*, *MaxPool* (in this example)
- torch.optim: for **Optimizers**
- torchvision: Handling **Datasets**

Numpy the basic scientific computing package used in customary.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Dataset

PyTorch basically provides MNIST Dataset and support download in running code!

In [ ]:
DATASET_DIR = './data' # path to download mnist dataset

TRAIN_DATASET = datasets.MNIST(DATASET_DIR,   # Dataset root path
                               train=True,    # Train data
                               download=True) # Download if not exist

TEST_DATASET = datasets.MNIST(DATASET_DIR,    # Dataset root path
                              train=False)    # Test data

#### Network

This is a simple convolution layer network.

# TODO
Implement the network as described in the comments.

In [ ]:
class Network(nn.Module):
    """ Simple neural network for classification MNIST
	Input:	N, 1, 28, 28
	Conv1:	5 x 5 kernel, 20 output feature, stride = 1, padding = 0
	ReLU
	Pool1:	2 x 2 (max pooling) 			
	Conv2:	5 x 5 kernel, 50 output feature, stride = 1, padding = 0
	ReLU
	Pool2:	2 x 2 (max pooling) 				
	Conv3:	? x ? kernel, 500 output features, stride = 1, padding = 0
	ReLU
	Fc1:	1 x 1, 10, stride = 1, padding = 0
	LogSoftmax -> 10 x N class probabilities
	"""
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, , , 1)
        self.conv2 = nn.Conv2d(, , , 1)
        self.conv3 = nn.Conv2d(, , , 1)
        self.fc1 = nn.Linear(500, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO: impelement foward
        return F.log_softmax(x, dim=1)

## Define Train and Test functions

In [ ]:
from typing import Tuple

def train(model, device, train_loader, optimizer) -> float:
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

    return loss.item()


def test(model, device, test_loader) -> Tuple[float, float, torch.Tensor]:
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, accuracy, output

In [ ]:
torch.manual_seed(42) # 42, THE ANSWER TO LIFE, THE UNIVERSE AND EVERYTHING

batch = 64            # batch size
lr = .01              # learning rate
epochs = 10


TRAIN_DATASET.transform = transforms.ToTensor()
train_loader = torch.utils.data.DataLoader(TRAIN_DATASET,
                                           batch_size=64,
                                           shuffle=True)

TEST_DATASET.transform = transforms.ToTensor()
test_loader = torch.utils.data.DataLoader(TEST_DATASET,
                                          batch_size=64,
                                          shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = Network().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer)
    test_loss, accuracy, _ = test(model, device, test_loader)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "mnist_cnn.pt")